# First home assignment Social Data Science

### Implement multiple linear regression by gradient descent
Your implementation should be based on numpy arrays. Your implementation can use functions from the numpy library.

Your function should have the signature
linear_regression_gd (X,y,learning_rate) and should return a tuple (mean_squared_error_of_solution, [list_of_optimum_parameters])
Here, X is a numpy array with the values of the explanatory variables, and y is an array with the dependent variable



In [1]:
import numpy as np

In [2]:
# https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html
# expect our input to be in the same shape

In [3]:
# https://stackoverflow.com/questions/8486294/how-to-add-an-extra-column-to-a-numpy-array

In [4]:
X = np.array([[1, 1], [1, 2], [2, 2], [2, 3]])
y = np.dot(X, np.array([[1], [2]])) + 3

In [5]:
# inspiration: https://towardsdatascience.com/gradient-descent-in-python-a0d07285742f
# X: numpy array with values of the explanatory variables
# y: array with dependent variables
def linear_regression_gd(X,y,learning_rate): 
    n_samples = (np.shape(X)[0])
    p = np.shape(X)[1] # p is the number of variables in the multiple regression
    b = np.ones((p+1,1)) #'guess' the initial values as zeros, +1 for the b_0 value
    X = np.insert(X, 0, values=1, axis=1) # add a column of ones s.t. y=X*b
    
    # now the gradient descent:
    for i in range(10000):
        prediction = np.dot(X,b)
        #print(prediction)
        d_b = (-1)*np.dot(np.transpose(X),(2*(y-prediction))) # derivative w.r.t. b
        #print(d_b)
        b = b - 1/n_samples * learning_rate*d_b # b = b - learning_rate*gradient(f)
        #print(b)

    #return (mean_squared_error_of_solution,list_of_optimum_parameters)
    return (np.mean((y-np.dot(X,b))**2), b) #np.mean((y-np.dot(X,b))**2), b)

In [6]:
linear_regression_gd(X,y,0.01) #... should be 3,1,2

(5.38399427584945e-14, array([[2.99999932],
        [1.00000052],
        [1.99999993]]))

### Extend your previous code to implement Stochastic Gradient Descent
Your function should have the signature
linear_regression_sgd (X,y,learning_rate, batch_size)

In [7]:
def linear_regression_sgd(X,y,learning_rate,batch_size):
    n_samples = (np.shape(X)[0])
    p = np.shape(X)[1] # p is the number of variables in the multiple regression
    b = np.ones((p+1,1)) #'guess' the initial values as zeros, +1 for the b_0 value
    X = np.insert(X, 0, values=1, axis=1) # add a column of ones s.t. y=X*b
    
    # now the stochastic gradient descent:
    for i in range(10000):
        # randomly select batch_size sample
        batch_ids = np.random.randint(np.shape(X)[0],size=batch_size)
        #print('batch size: ', batch_ids)
        X_batch = X[batch_ids,:]
        #print(X_batch)
        y_batch = y[batch_ids,:]
        #print(y_batch)
        
        # adjust with that batch
        prediction = np.dot(X_batch,b)
        #print(prediction)
        d_b = (-1)*np.dot(np.transpose(X_batch),(2*(y_batch-prediction))) # derivative w.r.t. b
        b = b - 1/n_samples * learning_rate*d_b # b = b - learning_rate*gradient(f)
    

    #return (mean_squared_error_of_solution,list_of_optimum_parameters)
    return ( np.mean((y-np.dot(X,b))**2) , b)

In [8]:
linear_regression_sgd(X,y,0.01,4)

(5.2000288087184836e-14, array([[2.99999932],
        [1.0000005 ],
        [1.99999995]]))

### Load the quality of Government Dataset 
from here: https://www.qogdata.pol.gu.se/data/qog_bas_cs_jan18.csv.
The data is described here
load it into a pandas dataframe and select the following columns:
"cname","wdi_lifexp","wdi_popden","gle_cgdpc","bti_acp", "bti_pdi", "fh_pair", "al_ethnic","al_language","al_religion","bti_aar","vdem_gender","bti_ci","bti_foe","wdi_araland", "wdi_forest"


In [12]:
import pandas as pd
data = pd.read_csv('qog_bas_cs_jan18.csv')
columnNames = ["cname","wdi_lifexp","wdi_popden","gle_cgdpc","bti_acp", "bti_pdi", "fh_pair", "al_ethnic","al_language","al_religion","bti_aar","vdem_gender","bti_ci","bti_foe","wdi_araland", "wdi_forest"]
data = data[columnNames]

### Compute the correlation of all other variables with the life expectancy (wdi_lifexp)

In [14]:
cor = data.corr()

In [15]:
cor["wdi_lifexp"]

wdi_lifexp     1.000000
wdi_popden     0.171756
gle_cgdpc      0.632859
bti_acp        0.449749
bti_pdi        0.181737
fh_pair        0.666925
al_ethnic     -0.555538
al_language   -0.550292
al_religion   -0.170274
bti_aar        0.136264
vdem_gender    0.431190
bti_ci        -0.463416
bti_foe        0.130552
wdi_araland    0.037243
wdi_forest     0.026779
Name: wdi_lifexp, dtype: float64

### Apply your own implementations  (GD and SGD)
To model the life expectancy from the population density and GDP per capita.
Compare the results with what you get from scikit learn OR statsmodel libraries.

For this subtask, just remove all countries with missing values in any of these three variables.

In [20]:
from sklearn import preprocessing
# extract relevant data for our case
XYData = data[['cname','wdi_popden','gle_cgdpc', 'wdi_lifexp']]

# preprocess: drop NaN, scale
XYData = XYData.dropna() 
X = XYData[['wdi_popden','gle_cgdpc']]
X = preprocessing.scale(X)
y = XYData[['wdi_lifexp']]
y = preprocessing.scale(y)

In [28]:
linear_regression_gd(X,y,0.01)

(0.5979993239144865, array([[2.94201082e-16],
        [3.94906609e-02],
        [6.24556484e-01]]))

In [29]:
linear_regression_sgd(X,y,0.01,100)

(0.5982986377928871, array([[-0.00585511],
        [ 0.04184107],
        [ 0.60794555]]))

In [30]:
from sklearn.linear_model import LinearRegression
LinearRegression().fit(X, y).intercept_, LinearRegression().fit(X, y).coef_

(array([2.89817631e-16]), array([[0.03949066, 0.62455648]]))

### Build regression models to model the life expectancy (wdi_lifexp) in this dataset
from all other mentioned variables.
You can use scikit learn and/or statsmodels libraries for this task.
Standardize variables and fill in missing values appropriately.
Compare linear regression, Ridge regression and Lasso using k-fold-cross validation
Test several parameters for the regularized regressions.


In [ ]:
# assumption: leave out cname

In [31]:
X = data[["wdi_popden","gle_cgdpc","bti_acp", "bti_pdi", "fh_pair", "al_ethnic","al_language","al_religion","bti_aar","vdem_gender","bti_ci","bti_foe","wdi_araland", "wdi_forest"]]
y = data[["wdi_lifexp"]]

In [32]:
# https://scikit-learn.org/stable/modules/impute.html
# https://scikit-learn.org/0.18/modules/preprocessing.html#imputation
import sklearn
from sklearn.preprocessing import Imputer
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
X = imp.fit_transform(X)
y = imp.fit_transform(y)

In [33]:
X = sklearn.preprocessing.scale(X)
y = sklearn.preprocessing.scale(y)

In [35]:
k=10

In [38]:
# https://scikit-learn.org/stable/modules/cross_validation.html
# https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html#sklearn.model_selection.cross_val_score
from sklearn.model_selection import ShuffleSplit, cross_val_score
cv = ShuffleSplit(n_splits = k, test_size=0.1)
lg = LinearRegression()
cross_val_score(lg,X,y,cv=cv)

array([0.6882664 , 0.79996008, 0.70077573, 0.31323931, 0.57288526,
       0.51966294, 0.59194857, 0.54073868, 0.4189656 , 0.6473073 ])

In [44]:
# ridge regression
from sklearn.linear_model import Ridge
lasso = Ridge()
cross_val_score(lasso,X,y,cv=cv)

array([0.24247205, 0.74391843, 0.71124449, 0.2861093 , 0.4863119 ,
       0.65838797, 0.71009055, 0.56301846, 0.66896358, 0.68431846])

In [76]:
cross_val_score(Ridge(alpha=0),X,y,cv=cv) # linear Regression

array([0.38713716, 0.64992386, 0.5614686 , 0.62753713, 0.75392513,
       0.50683957, 0.75905328, 0.35099827, 0.61660031, 0.28368014])

In [95]:
cross_val_score(Ridge(alpha=100000),X,y,cv=cv)  # coefficents go to zero

array([-1.55682328e-01, -1.10851528e-01,  4.31315191e-03, -3.05177322e-02,
       -5.32554858e-02, -3.98832135e-03, -2.46758161e-02, -2.50289936e-05,
       -1.07688815e-01, -2.60768162e-03])

In [100]:
cross_val_score(Ridge(alpha=1e-10),X,y,cv=cv)

array([0.70041334, 0.48501392, 0.66281824, 0.58699116, 0.617011  ,
       0.6629976 , 0.6125895 , 0.58041788, 0.60254069, 0.71888772])

In [53]:
# lasso regression
from sklearn.linear_model import Lasso
lasso = Lasso()
cross_val_score(lasso,X,y,cv=cv)

array([-7.32652397e-03, -2.12285296e-01, -3.16084234e-02, -1.53736963e-04,
       -8.24755112e-02, -5.89595520e-02, -2.46081627e-01, -3.82267872e-03,
       -1.18363905e-01, -1.75649063e-02])

In [103]:
cross_val_score(Lasso(alpha=1e-15),X,y,cv=cv)

array([0.58376861, 0.61597261, 0.68833639, 0.51883046, 0.77254637,
       0.67285183, 0.64478344, 0.48576241, 0.59639056, 0.55148189])

In [106]:
cross_val_score(Lasso(alpha=1e-5),X,y,cv=cv)

array([0.21386891, 0.81958361, 0.5454186 , 0.61071232, 0.1717096 ,
       0.57845261, 0.51944198, 0.64159504, 0.683813  , 0.74992033])

In [107]:
cross_val_score(Lasso(alpha=1e-10),X,y,cv=cv)

array([0.1865865 , 0.5055625 , 0.72825153, 0.59185169, 0.78917682,
       0.59664045, 0.59222486, 0.46265087, 0.71035191, 0.83755993])

In [108]:
cross_val_score(Lasso(alpha=1e-12),X,y,cv=cv)

array([0.33812857, 0.53014303, 0.51518985, 0.46217371, 0.54196854,
       0.63625363, 0.50313216, 0.68570864, 0.52509479, 0.52472505])

### Implement Forward and Backward Selection algorithms
And apply it to the (given subset of variables of the) Quality of Government dataset.
Compare the results of Forward and Backward selection with each other.


In [168]:
def preprocess(X, features):
    transformed = X[features]
    transformed = imp.fit_transform(transformed)
    transformed = sklearn.preprocessing.scale(transformed)
    return transformed

In [169]:
def forwardSelection(X,y,available_features, current_features):
    result = []
    while(len(available_features)>0):
        #preprocess y
        y = imp.fit_transform(y)
        y = sklearn.preprocessing.scale(y)

        highestScore= -500 #smallestError = 10000
        bestFeature = "wdi_popden"
        for feature in available_features:
            current_features.append(feature)

            # preprocess X[current_features]
            transformed = preprocess(X,current_features)

            #curError = linear_regression_gd(transformed,y, alpha)[0]
            curScore = np.mean(cross_val_score(lg,transformed,y,cv=cv))
            #print(curScore)
            #print(curError)
            current_features.remove(feature)
            #print(curError<smallestError)
            #if (curError<smallestError):
            if (curScore>highestScore):
                bestFeature = feature
                #smallestError = curError
                highestScore=curScore
        
        result.append(highestScore)
        available_features.remove(bestFeature)
        current_features.append(bestFeature)
        #print(current_features)
        print('current features:', current_features)
        #print('according error:', smallestError)
        print('according score:', highestScore)
    maxIndex = np.argmax(result)
    print(result)
    print(maxIndex)
    return current_features[:maxIndex+1]

In [170]:
## K-fold cross-validation (10%, 90%)

In [172]:
features = ["wdi_popden","gle_cgdpc","bti_acp", "bti_pdi", "fh_pair", "al_ethnic","al_language","al_religion","bti_aar","vdem_gender","bti_ci","bti_foe","wdi_araland", "wdi_forest"]
forwardSelection(data[features], data[['wdi_lifexp']],features,[])

current features: ['fh_pair']
according score: 0.29633143513589927
current features: ['fh_pair', 'bti_aar']
according score: 0.5523356309614205
current features: ['fh_pair', 'bti_aar', 'gle_cgdpc']
according score: 0.5724678575271285
current features: ['fh_pair', 'bti_aar', 'gle_cgdpc', 'al_ethnic']
according score: 0.5922330451717627
current features: ['fh_pair', 'bti_aar', 'gle_cgdpc', 'al_ethnic', 'al_language']
according score: 0.6496532764376307
current features: ['fh_pair', 'bti_aar', 'gle_cgdpc', 'al_ethnic', 'al_language', 'bti_pdi']
according score: 0.6608953741951082
current features: ['fh_pair', 'bti_aar', 'gle_cgdpc', 'al_ethnic', 'al_language', 'bti_pdi', 'bti_ci']
according score: 0.638442759593848
current features: ['fh_pair', 'bti_aar', 'gle_cgdpc', 'al_ethnic', 'al_language', 'bti_pdi', 'bti_ci', 'bti_foe']
according score: 0.683577175346125
current features: ['fh_pair', 'bti_aar', 'gle_cgdpc', 'al_ethnic', 'al_language', 'bti_pdi', 'bti_ci', 'bti_foe', 'bti_acp']
acco

['fh_pair',
 'bti_aar',
 'gle_cgdpc',
 'al_ethnic',
 'al_language',
 'bti_pdi',
 'bti_ci',
 'bti_foe',
 'bti_acp',
 'vdem_gender']

In [177]:
def backwardSelection(X,y,removed_features, current_features):
    result = []
    #preprocess y
    y = imp.fit_transform(y)
    y = sklearn.preprocessing.scale(y)
    # backward selection
    while(len(current_features)>1):
        lowestScore= 500 #smallestError = 10000
        worstFeature = "wdi_popden"
        
        for feature in current_features:
            current_features.remove(feature)
            # preprocess X[current_features]
            transformed = preprocess(X, current_features)
            # calculate score for current feature
            curScore = np.mean(cross_val_score(lg,transformed,y,cv=cv))
            current_features.append(feature)
            # find lowest score
            if (curScore<lowestScore):
                worstFeature = feature
                lowestScore = curScore
        result.append(np.mean(cross_val_score(lg,preprocess(X,current_features),y,cv=cv)))
        current_features.remove(worstFeature)
        removed_features.append(worstFeature)
        print('current features:', current_features)
        #print('according error:', smallestError)
        print('worst score before:', lowestScore)
    maxIndex = np.argmax(result)
    print(result)
    print(maxIndex)
    print(removed_features)
    return removed_features[maxIndex:]

In [183]:
features = ["wdi_popden","gle_cgdpc","bti_acp", "bti_pdi", "fh_pair", "al_ethnic","al_language","al_religion","bti_aar","vdem_gender","bti_ci","bti_foe","wdi_araland", "wdi_forest"]
backwardSelection(data[features], data[['wdi_lifexp']],[],features)

current features: ['gle_cgdpc', 'bti_pdi', 'al_ethnic', 'al_religion', 'vdem_gender', 'bti_foe', 'wdi_forest', 'bti_acp', 'al_language', 'bti_ci', 'wdi_popden', 'bti_aar', 'fh_pair']
worst score before: -0.04627996323580156
current features: ['bti_pdi', 'al_religion', 'bti_foe', 'bti_acp', 'bti_ci', 'bti_aar', 'gle_cgdpc', 'vdem_gender', 'al_language', 'fh_pair', 'wdi_forest', 'al_ethnic']
worst score before: 0.3255208769220078
current features: ['al_religion', 'bti_acp', 'bti_aar', 'vdem_gender', 'fh_pair', 'al_ethnic', 'bti_foe', 'gle_cgdpc', 'wdi_forest', 'bti_ci', 'al_language']
worst score before: 0.47706751942742825
current features: ['bti_acp', 'vdem_gender', 'al_ethnic', 'gle_cgdpc', 'bti_ci', 'al_religion', 'wdi_forest', 'bti_aar', 'al_language', 'bti_foe']
worst score before: 0.48036916912875105
current features: ['vdem_gender', 'gle_cgdpc', 'al_religion', 'bti_aar', 'bti_foe', 'wdi_forest', 'bti_acp', 'al_language', 'bti_ci']
worst score before: 0.486207572908787
current fea

['bti_pdi',
 'fh_pair',
 'al_ethnic',
 'al_religion',
 'gle_cgdpc',
 'vdem_gender',
 'al_language',
 'bti_foe',
 'bti_aar',
 'bti_acp',
 'wdi_forest']